In [1]:
import cv2
import imghdr
import tensorflow as tf
import os

In [2]:
data_dir = 'Training' 

In [4]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [5]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [6]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
data = tf.keras.utils.image_dataset_from_directory('Training')

Found 5711 files belonging to 4 classes.


preprocessing

In [8]:
import cv2
X_train = []
y_train = []
X_test = []
y_test = []
image_size = 224
labels = ['glioma','notumor','meningioma','pituitary']
for i in labels:
    folderPath = os.path.join('Training',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('Testing',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_test.append(img)
        y_test.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
print(tf.config.list_physical_devices('GPU'))

[]


In [10]:
from sklearn.utils import shuffle
X_train,y_train=shuffle(X_train,y_train,random_state=101)
X_train.shape

(5711, 224, 224, 3)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=.1,random_state=101)

In [11]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

In [12]:
y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

callback(early stop condition)

In [13]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

In [18]:
Model = Sequential()
Model.add(Conv2D(32,(3,3),activation="relu", input_shape=(224,224,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))
Model.add(Conv2D(64,(3,3),  activation="relu",padding="same"))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))
Model.add(Conv2D(128,(3,3), activation="relu",padding="same"))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((3,3)))
Model.add(Conv2D(256,(3,3), activation="relu",padding="same"))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((3,3)))
Model.add(Conv2D(512,(3,3), activation="relu",padding="same"))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((3,3)))
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dense(4,activation="sigmoid"))

In [19]:
Model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(lr=3e-5) ,metrics='accuracy')
Model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 111, 111, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 111, 111, 64)     256       
 hNormalization)                                                 
                                                      

In [20]:
history = Model.fit(X_train,y_train, epochs=50, validation_split=.2, callbacks=[tensorboard_callback])

Epoch 1/50
 46/143 [========>.....................] - ETA: 3:52 - loss: 1.2902 - accuracy: 0.6433

KeyboardInterrupt: 

In [ ]:
score_cnn = Model.evaluate(X_test,y_test, verbose = 0)

print('Test loss:', score_cnn[0]) 
print('Test accuracy:', score_cnn[1])

In [ ]:
# Plot the loss  curves for training and validation 
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Plot the loss  accuracy for training and validation 
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
pred = Model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)
accuracy = np.sum(pred==y_test_new)/len(pred)
print("Accuracy on testing dataset: {:.2f}%".format(accuracy*100))

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
Model.save(os.path.join('models','brainclassifier_cnn.h5'))

In [ ]:
new_model = load_model('models/brainclassifier_cnn.h5')

img = cv2.imread('pt.jpg')
plt.imshow(img)
plt.show()

resize = tf.image.resize(img, (150,150))
plt.imshow(resize.numpy().astype(int))
plt.show()

yhat = new_model.predict(np.expand_dims(resize, 0))

yhat

output = { 0:'glioma_tumor',1:'no_tumor',2:'meningioma_tumor',3:'pituitary_tumor'}
print("Predicted :- ",output[np.argmax(yhat)])

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import random
import matplotlib.cm as cm

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:

def print_confusion_matrix(confusion_matrix, class_names, figsize = (12, 12), fontsize=14):
    group_counts = ["{0:0.0f}".format(value) for value in
                    confusion_matrix.flatten()]

    group_percentages = ["{0:.2%}".format(value) for value in
                         confusion_matrix.flatten() / np.sum(confusion_matrix)]

    labels = [f"{v2}\n{v3}" for v2, v3 in
              zip(group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(len(class_names), len(class_names))

    
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=labels, fmt='', cmap='Blues')

    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Note that due to returning the created figure object, when this funciton is called in a notebook
    # the figure willl be printed twice. To prevent this, either append ; to your function call, or
    # modify the function by commenting out this return expression.
    return fig

In [ ]:

test_predictions = Model.predict(X_test)
ground_truth_class_ids = y_test.argmax(axis=1)

# take the argmax for each set of prediction scores
# to return the class id of the highest confidence prediction
top_pred_ids = test_predictions.argmax(axis=1)
conf_mat = confusion_matrix(ground_truth_class_ids, top_pred_ids)
figure = print_confusion_matrix(conf_mat, labels)
plt.title('Confusion Matrix')
plt.show()